# Relaxation effects ($ T_2 $)

In this notebook we demonstrate the use of additional $T_2$ relaxation effects applied on top of an existing TissueModel.

## Setting up the dmipy model

We demonstrate this using a simple model from dmipy

In [ ]:
from dmipy.core.modeling_framework import MultiCompartmentModel
from dmipy.signal_models.gaussian_models import G1Ball

from microtool.dmipy import DmipyTissueModel
from microtool.tissue_model import RelaxedMultiTissueModel

# setting up a simple model
simple_ball = G1Ball(lambda_iso=1.7e-9)

# This step is unnecessary
dmipy_model = MultiCompartmentModel([simple_ball])

microtool_model = DmipyTissueModel(dmipy_model)

relaxation_time = 100.0
model = RelaxedMultiTissueModel([microtool_model],[1.0], [relaxation_time])
print(model)

## Setting up the acquisition scheme

In [ ]:
from microtool.dmipy import convert_dmipy_scheme2diffusion_scheme

from microtool.utils.saved_schemes import alexander2008_optimized_directions


acq_scheme = alexander2008_optimized_directions([2,8,16])
acq_wrapped = convert_dmipy_scheme2diffusion_scheme(acq_scheme)

print(acq_wrapped)

## Generating signal

In [ ]:
# relaxation signal
model(acq_wrapped)

## Optimizing the scheme with respect to $ T_2 $.

In [ ]:
from microtool.optimize import optimize_scheme

scheme, _ = optimize_scheme(acq_wrapped, model, noise_variance=0.02, method='differential_evolution')

## Fitting the model

## Validation of scheme optimization through Monte Carlo simulations

In [ ]:
from microtool.monte_carlo.simulation import MonteCarloSimulation
from scipy import stats

# loc is mean and scale is standard deviation
noise_distribution = stats.norm(loc=0, scale=0.02)

simulation = MonteCarloSimulation(scheme,model,noise_distribution,n_sim=100)

simulation.run()
simulation.save